![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/23.0.Medical_Question_Answering.ipynb)


# **Medical Question Answering**

**Welcoming BioGPT (Generative Pre-Trained Transformer For Biomedical Text Generation and Mining) to Spark NLP**

BioGPT is a domain-specific generative pre-trained Transformer language model for biomedical text generation and mining. BioGPT follows the Transformer language model backbone, and is pre-trained on 15M PubMed abstracts from scratch. Experiments demonstrate that BioGPT achieves better performance compared with baseline methods and other well-performing methods across all the tasks. Read more at [the official paper.](https://arxiv.org/abs/2210.10341)

We ported BioGPT (`BioGPT-QA-PubMedQA-BioGPT`) into Spark NLP for Healthcare with better inference speed and memory optimization, that enable us to get 20x faster results with less memory compared to the HuggingFace version.


## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs==5.1.0

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/5.1.0.spark_nlp_for_healthcare.json
🚨 Outdated OCR Secrets in license file. Version=4.4.4 but should be Version=5.0.1
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.1.0, 💊Spark-Healthcare==5.1.0, running on ⚡ PySpark==3.1.2


In [ ]:
spark

In [ ]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

# 🔎 MODELS

<div align="center">

| **Index** | **Question Answer Models**        |
|---------------|----------------------|
| 1        | [medical_qa_biogpt](https://nlp.johnsnowlabs.com/2023/05/17/medical_qa_biogpt_en.html)     |
| 2          | [flan_t5_base_jsl_qa](https://nlp.johnsnowlabs.com/2023/05/15/flan_t5_base_jsl_qa_en.html)       |
| 3          | [clinical_notes_qa_large](https://nlp.johnsnowlabs.com/2023/07/06/clinical_notes_qa_base_en.html)       |
| 4          | [clinical_notes_qa_base](https://nlp.johnsnowlabs.com/2023/07/06/clinical_notes_qa_large_en.html)       |
| 5          | [clinical_notes_qa_base_onnx](https://nlp.johnsnowlabs.com/2023/08/17/clinical_notes_qa_base_onnx_en.html)|
| 6          | [clinical_notes_qa_large_onnx](https://nlp.johnsnowlabs.com/2023/08/17/clinical_notes_qa_large_onnx_en.html)|
|||


</div>

## [medical_qa_biogpt](https://nlp.johnsnowlabs.com/2023/03/09/medical_qa_biogpt_en.html)
This model has been trained with medical documents and can generate two types of answers, `short` and `long`.

In [ ]:
paper_abstract = [
    "We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories.",
    "To evaluate the degree to which histologic chorioamnionitis, a frequent finding in placentas submitted for histopathologic evaluation, correlates with clinical indicators of infection in the mother. A retrospective review was performed on 52 cases with a histologic diagnosis of acute chorioamnionitis from 2,051 deliveries at University Hospital, Newark, from January 2003 to July 2003. Third-trimester placentas without histologic chorioamnionitis (n = 52) served as controls. Cases and controls were selected sequentially. Maternal medical records were reviewed for indicators of maternal infection. Histologic chorioamnionitis was significantly associated with the usage of antibiotics (p = 0.0095) and a higher mean white blood cell count (p = 0.018). The presence of 1 or more clinical indicators was significantly associated with the presence of histologic chorioamnionitis (p = 0.019).",
    "In patients with Los Angeles (LA) grade C or D oesophagitis, a positive relationship has been established between the duration of intragastric acid suppression and healing.AIM: To determine whether there is an apparent optimal time of intragastric acid suppression for maximal healing of reflux oesophagitis. Post hoc analysis of data from a proof-of-concept, double-blind, randomized study of 134 adult patients treated with esomeprazole (10 or 40 mg od for 4 weeks) for LA grade C or D oesophagitis. A curve was fitted to pooled 24-h intragastric pH (day 5) and endoscopically assessed healing (4 weeks) data using piecewise quadratic logistic regression. Maximal reflux oesophagitis healing rates were achieved when intragastric pH>4 was achieved for approximately 50-70% (12-17 h) of the 24-h period. Acid suppression above this threshold did not yield further increases in healing rates."
]

question = [
    "Transgastric endoscopic splenectomy: is it possible?",
    "Does histologic chorioamnionitis correspond to clinical chorioamnionitis?",
    "Is there an optimal time of acid suppression for maximal healing?"
]

data = spark.createDataFrame(
    [
        [paper_abstract[0],  question[0]],
        [paper_abstract[1],  question[1]],
        [paper_abstract[2],  question[2]],
    ]
).toDF("context","question")

data.show(truncate = 60)

+------------------------------------------------------------+------------------------------------------------------------+
|                                                     context|                                                    question|
+------------------------------------------------------------+------------------------------------------------------------+
|We have previously reported the feasibility of diagnostic...|        Transgastric endoscopic splenectomy: is it possible?|
|To evaluate the degree to which histologic chorioamnionit...|Does histologic chorioamnionitis correspond to clinical c...|
|In patients with Los Angeles (LA) grade C or D oesophagit...|Is there an optimal time of acid suppression for maximal ...|
+------------------------------------------------------------+------------------------------------------------------------+



### **Short answer**

 `Short` answers usually consist of brief and concise responses such as "yes" or "no", and they are typically given as yes/no responses to do/does, is/are, or other question words.

In [ ]:
document_assembler = nlp.MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa =medical.MedicalQuestionAnswering.pretrained("medical_qa_biogpt","en","clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setOutputCol("answer")\
    .setMaxNewTokens(30)\
    .setTopK(1)\
    .setQuestionType("short")

pipeline = nlp.Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([["",""]]).toDF("question", "context")

model = pipeline.fit(empty_data)

medical_qa_biogpt download started this may take some time.
[OK!]


In [ ]:
model.transform(data)\
    .selectExpr("document_question.result as Question", "answer.result as Short_Answer")\
    .show(truncate=False)

+---------------------------------------------------------------------------+------------+
|Question                                                                   |Short_Answer|
+---------------------------------------------------------------------------+------------+
|[Transgastric endoscopic splenectomy: is it possible?]                     |[yes]       |
|[Does histologic chorioamnionitis correspond to clinical chorioamnionitis?]|[yes]       |
|[Is there an optimal time of acid suppression for maximal healing?]        |[no]        |
+---------------------------------------------------------------------------+------------+



**LightPipelines**

In [ ]:
context ="We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories."
question = "Transgastric endoscopic splenectomy: is it possible?"

light_model = nlp.LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

[{'document_question': ['Transgastric endoscopic splenectomy: is it possible?'],
  'document_context': ['We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity 

In [ ]:
light_result[0]["answer"]

['yes']

### **Long answer**
A `long` answer is a response that goes into more detail and provides a more comprehensive explanation than a `short` answer. `Long` answers typically involve more elaboration, examples, and supporting evidence to provide a thorough understanding of a particular topic or question.

In [ ]:
document_assembler = nlp.MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa = medical.MedicalQuestionAnswering.pretrained("medical_qa_biogpt","en","clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setOutputCol("answer")\
    .setMaxNewTokens(30)\
    .setTopK(1)\
    .setQuestionType("long") # "short"

pipeline = nlp.Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

medical_qa_biogpt download started this may take some time.
[OK!]


In [ ]:
model.transform(data)\
  .selectExpr("document_question.result as Question", "answer.result as Long_Answer")\
  .show(truncate=False)

+---------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Question                                                                   |Long_Answer                                                                                                                                                                                             |
+---------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[Transgastric endoscopic splenectomy: is it possible?]                     |[per - oral transgastric splenectomy was technically feasible in a porcine model. furt

**LightPipelines**

In [ ]:
context ="We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories."
question = "Transgastric endoscopic splenectomy: is it possible?"

light_model = nlp.LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

[{'document_question': ['Transgastric endoscopic splenectomy: is it possible?'],
  'document_context': ['We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity 

In [ ]:
light_result[0]["answer"]

['per - oral transgastric splenectomy was technically feasible in a porcine model. further studies are necessary to determine the safety and efficacy of this procedure in']

## clinical_notes_qa_base_onnx

In [ ]:
document_assembler = nlp.MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa  = medical.MedicalQuestionAnswering().pretrained("clinical_notes_qa_base_onnx", "en", "clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setCustomPrompt("Context: {context} \n Question: {question} \n Answer: ")\
    .setOutputCol("answer")\

pipeline = nlp.Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

clinical_notes_qa_base_onnx download started this may take some time.
[OK!]


In [ ]:
note_text = '''
  Patient with a past medical history of hypertension for 15 years.
  (Medical Transcription Sample Report)\nHISTORY OF PRESENT ILLNESS:
  The patient is a 74-year-old white woman who has a past medical history of hypertension for 15 years, history of CVA with no residual hemiparesis and uterine cancer with pulmonary metastases, who presented for evaluation of recent worsening of the hypertension. According to the patient, she had stable blood pressure for the past 12-15 years on 10 mg of lisinopril.
  '''

question = "What is the primary issue reported by patient?"

data = spark.createDataFrame([[question, note_text]]).toDF("question", "context")

result = pipeline.fit(data).transform(data)

In [ ]:
result.selectExpr("document_question.result as Question", "answer.result as Answer")\
  .show(truncate=False)

+------------------------------------------------+------------------------------------------------------------+
|Question                                        |Answer                                                      |
+------------------------------------------------+------------------------------------------------------------+
|[What is the primary issue reported by patient?]|[The primary issue reported by the patient is hypertension.]|
+------------------------------------------------+------------------------------------------------------------+



In [ ]:
context ="I am asked to see the patient today with ongoing issues around her diabetic control. We have been fairly aggressively, downwardly adjusting her insulins, both the Lantus insulin, which we had been giving at night as well as her sliding scale Humalog insulin prior to meals. Despite frequent decreases in her insulin regimen, she continues to have somewhat low blood glucoses, most notably in the morning when the glucoses have been in the 70s despite decreasing her Lantus insulin from around 84 units down to 60 units, which is a considerable change. What I cannot explain is why her glucoses have not really climbed at all despite the decrease in insulin. The staff reports to me that her appetite is good and that she is eating as well as ever. I talked to Anna today. She feels a little fatigued. Otherwise, she is doing well."
question = "How much has the Lantus insulin dosage been reduced?"

pipeline

light_model = nlp.LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

[{'document_question': ['How much has the Lantus insulin dosage been reduced?'],
  'document_context': ['I am asked to see the patient today with ongoing issues around her diabetic control. We have been fairly aggressively, downwardly adjusting her insulins, both the Lantus insulin, which we had been giving at night as well as her sliding scale Humalog insulin prior to meals. Despite frequent decreases in her insulin regimen, she continues to have somewhat low blood glucoses, most notably in the morning when the glucoses have been in the 70s despite decreasing her Lantus insulin from around 84 units down to 60 units, which is a considerable change. What I cannot explain is why her glucoses have not really climbed at all despite the decrease in insulin. The staff reports to me that her appetite is good and that she is eating as well as ever. I talked to Anna today. She feels a little fatigued. Otherwise, she is doing well.'],
  'answer': ['The Lantus insulin dosage has been reduced from

## clinical_notes_qa_large_onnx

In [ ]:
document_assembler = nlp.MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa  = medical.MedicalQuestionAnswering().pretrained("clinical_notes_qa_large_onnx", "en", "clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setCustomPrompt("Context: {context} \n Question: {question} \n Answer: ")\
    .setOutputCol("answer")

pipeline = nlp.Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

clinical_notes_qa_large_onnx download started this may take some time.
[OK!]


In [ ]:
note_text = '''
  Patient with a past medical history of hypertension for 15 years.
  (Medical Transcription Sample Report)\nHISTORY OF PRESENT ILLNESS:
  The patient is a 74-year-old white woman who has a past medical history of hypertension for 15 years, history of CVA with no residual hemiparesis and uterine cancer with pulmonary metastases, who presented for evaluation of recent worsening of the hypertension. According to the patient, she had stable blood pressure for the past 12-15 years on 10 mg of lisinopril.
  '''

question = "What is the primary issue reported by patient?"

data = spark.createDataFrame([[question, note_text]]).toDF("question", "context")

result = pipeline.fit(data).transform(data)

In [ ]:
result.selectExpr("document_question.result as Question", "answer.result as Answer")\
  .show(truncate=False)

+------------------------------------------------+--------------------------------------------------------------------------------+
|Question                                        |Answer                                                                          |
+------------------------------------------------+--------------------------------------------------------------------------------+
|[What is the primary issue reported by patient?]|[The primary issue reported by patient is recent worsening of the hypertension.]|
+------------------------------------------------+--------------------------------------------------------------------------------+



In [ ]:
context ="his is a 14-month-old with history of chronic recurrent episodes of otitis media, totalling 6 bouts, requiring antibiotics since birth. There is also associated chronic nasal congestion. There had been no bouts of spontaneous tympanic membrane perforation, but there had been elevations of temperature up to 102 during the acute infection. He is being admitted at this time for myringotomy and tube insertion under general facemask anesthesia."
question = "How many bouts of otitis media has the patient experienced?"

light_model = nlp.LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

[{'document_question': ['How many bouts of otitis media has the patient experienced?'],
  'document_context': ['his is a 14-month-old with history of chronic recurrent episodes of otitis media, totalling 6 bouts, requiring antibiotics since birth. There is also associated chronic nasal congestion. There had been no bouts of spontaneous tympanic membrane perforation, but there had been elevations of temperature up to 102 during the acute infection. He is being admitted at this time for myringotomy and tube insertion under general facemask anesthesia.'],
  'answer': ['The patient has experienced 6 bouts of otitis media.']}]